In [ ]:
# # !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [ ]:
# !kaggle datasets download -d salader/dogs-vs-cats

 99% 1.06G/1.06G [00:07<00:00, 141MB/s]
100% 1.06G/1.06G [00:07<00:00, 162MB/s]


In [1]:
import zipfile
zip_ref = zipfile.ZipFile('/content/monkeypox.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import keras_tuner as kt
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns

In [3]:
# conv_base = InceptionV3(
#     weights='imagenet',
#     include_top = False,
#     input_shape=(224,224,3)
# )

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Reduce Learning Rate

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,  # Reduce learning rate by a factor of 0.2
    patience=3,  # Wait 3 epochs before reducing
    min_lr=1e-6  # Minimum learning rate
)

# Alternatively, use Cosine Decay or Exponential Decay for smooth learning rate reduction:

In [ ]:

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

In [5]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Load base model
conv_base = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

# Unfreeze top 30 layers of InceptionV3
for layer in conv_base.layers[-30:]:
    layer.trainable = True

# Freeze all layers initially
# conv_base.trainable = False

# Build model
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())  # ✅ THIS LINE FIXES THE SHAPE
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification




In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,327,585 (85.17 MB)

 Trainable params: 22,293,153 (85.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [7]:
# conv_base.trainable = False

In [8]:
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,               # always normalize
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply only normalization to validation
val_datagen = ImageDataGenerator(rescale=1./255)
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/Fold1/Fold1/Fold1/Train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/Fold1/Fold1/Fold1/Val',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

Found 2142 files belonging to 2 classes.
Found 420 files belonging to 2 classes.


# Below code is with the augmentation method

In [ ]:

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras imoport layers

# Define data augmentation as a keras Sequential model
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),  # Increase rotation
    layers.RandomZoom(0.3),
    layers.RandomTranslation(0.2, 0.2),
    layers.RandomContrast(0.2),  # Add contrast adjustment
    layers.RandomBrightness(0.2)  # Add brightness adjustment
])
# Load datasets from your directories (with your exact paths)
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/Fold1/Fold1/Fold1/Train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/Fold1/Fold1/Fold1/Val',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224)
)

# Function to apply augmentation and normalization to training data
def process_train(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    image = data_augmentation(image)            # Apply augmentation
    return image, label

# Function to normalize validation data only
def process_val(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image, label

# Apply the processing functions
train_ds = train_ds.map(process_train, num_parallel_calls=tf.data.AUTOTUNE)
validation_ds = validation_ds.map(process_val, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetch for performance
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
validation_ds = validation_ds.prefetch(tf.data.AUTOTUNE)

# model with hp tuning

def build_model(hp):
    model = Sequential()
    model.add(conv_base)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(
        units=hp.Int('units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    directory='tuner_dir',
    project_name='inceptionv3_tuning'
)
tuner.search(train_ds, validation_data=validation_ds, epochs=20)
best_model = tuner.get_best_models(num_models=1)[0]

In [10]:


# model.compile(
#     optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )



In [ ]:
history = model.fit(train_ds,epochs=20,validation_data=validation_ds)

Epoch 1/20
14/67 ━━━━━━━━━━━━━━━━━━━━ 14:01 16s/step - accuracy: 0.6053 - loss: 0.6710

 ## Compute additional metrics like AUC-ROC or confusion matrix to better understand model performance:

In [ ]:
y_true = []
y_pred_probs = []
for images, labels in validation_ds:
    preds = model.predict(images).flatten()
    y_true.extend(labels.numpy())
    y_pred_probs.extend(preds)

auc = roc_auc_score(y_true, y_pred_probs)
print(f"AUC-ROC: {auc:.4f}")

cm = confusion_matrix(y_true, (np.array(y_pred_probs) > 0.5).astype(int))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.show()

#### Visualize Training Curves
#### Your current plotting code is fine, but ensure you're monitoring for overfitting. If the validation loss increases while training loss decreases, the model is overfitting.

In [ ]:
def smooth_curve(points, factor=0.8):
    smoothed = []
    for point in points:
        if smoothed:
            smoothed.append(smoothed[-1] * factor + point * (1 - factor))
        else:
            smoothed.append(point)
    return smoothed

plt.plot(smooth_curve(history.history['accuracy']), color='red', label='train')
plt.plot(smooth_curve(history.history['val_accuracy']), color='blue', label='validation')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(smooth_curve(history.history['loss']), color='red', label='train')
plt.plot(smooth_curve(history.history['val_loss']), color='blue', label='validation')
plt.title('Model Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_true = []
y_pred = []

for images, labels in validation_ds:
    preds = model.predict(images).flatten()
    pred_labels = (preds > 0.5).astype(int)
    y_true.extend(labels.numpy())
    y_pred.extend(pred_labels)
class_name=["monkeypox","others"]
class_names = validation_ds.class_names  # ['monkeypox', 'others']

print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
# Load the Validation Dataset

from tensorflow.keras.utils import image_dataset_from_directory

val_dir = '/content/Fold1/Fold1/Fold1/Test'

val_ds = image_dataset_from_directory(
    directory=val_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)


In [ ]:
# Normalize the Images (like you did for training)

def preprocess(image, label):
    image = tensorflow.cast(image / 255.0, tensorflow.float32)
    return image, label

val_ds = val_ds.map(preprocess)


In [ ]:
loss, accuracy = model.evaluate(val_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [78]:
import os
from collections import Counter

train_dir = "/content/Fold1/Fold1/Fold1/Train"
print("Train class counts:")
for label in os.listdir(train_dir):
    count = len(os.listdir(os.path.join(train_dir, label)))
    print(f"{label}: {count}")


Train class counts:
Others: 1162
Monkeypox: 980
